In [1]:
import pandas as pd
df = pd.read_csv("articles_paragraphs.csv")

In [2]:
df_esp = df[df['language_code'] =='es'].reset_index()

In [5]:
import pandas as pd
import string
import spacy

# load spacy nlp model
nlp = spacy.load('es_core_news_sm')

# define function for pre-processing and tokenization
def preprocess_text_lemma(text):
    # lowercase
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # lemmatize
    doc = nlp(text)
    lemmatized_text = [token.lemma_ for token in doc]
    # remove stopwords and short words
    stopwords = spacy.lang.es.stop_words.STOP_WORDS
    tokens = [token for token in lemmatized_text if token not in stopwords and len(token) > 2]
    return tokens

# apply pre-processing and tokenization to the 'content' column of each row
tokenized_paragraphs_lemma = []
for paragraph in df_esp['content']:
    tokens = preprocess_text_lemma(paragraph)
    tokenized_paragraphs_lemma.append(tokens)

# print the resulting list of lists of tokens
print(tokenized_paragraphs_lemma)

[['opinión', 'frente', 'inteligencia', 'artificial', 'blog', 'llamar', 'cine', 'televisión', 'comunicación', 'masivo', 'empapar', 'historia', 'personaje', 'relacionado', 'construir', 'imaginario', 'frente', 'tema', 'tomar', 'ejemplo', 'universo', 'cinematográfico', 'marvel', 'studio', 'presentar', 'inteligencia', 'artificial', 'visión', 'superhéroir', 'cuyo', 'mente', 'poseer', 'resto', 'jarvis', 'asistente', 'personal', 'virtual', 'tony', 'stark', 'contraparte', 'infame', 'ultrón', 'robot', 'cuyo', 'objetivo', 'exterminar', 'raza', 'humano'], ['contraste', 'realidad', 'pensar', 'sistema', 'siri', 'alexa', 'presente', 'primordialmente', 'celular', 'asistirno', 'controversial', 'cámara', 'reconocimiento', 'facial', 'implementada', 'china', 'cuyo', 'generar', 'revuelta', 'hong', 'kong', 'sentir', 'tipo', 'tecnología', 'llegar', 'vulnerar', 'privacidad', 'población'], ['mente', 'tarea', 'preguntar él', 'cliente', 'amigo', 'familiar', 'opinión', 'frente', 'tecnología', 'opinión', 'recalcar

In [6]:
import gensim
import numpy as np


# Train Word2Vec model
lemmaModel = gensim.models.Word2Vec(tokenized_paragraphs_lemma,vector_size=40, window=15, min_count=2)

# Calculate the meaning vector per paragraph
paragraph_vectors_lemma = []
for paragraph_tokens in tokenized_paragraphs_lemma:
    vectors = []
    for token in paragraph_tokens:
        if token in lemmaModel.wv.key_to_index:
            vectors.append(lemmaModel.wv[token])
    if len(vectors) > 0:
        paragraph_vectors_lemma.append(np.mean(vectors, axis=0))
    else:
        paragraph_vectors_lemma.append(np.zeros(lemmaModel.vector_size))

#print(paragraph_vectors_lemma[383])

In [7]:
df_esp['vector'] = paragraph_vectors_lemma

In [8]:
import numpy as np
from gensim.models import KeyedVectors


def cosine_similarity_list(vectors_list, query_vector):
    #Compute the cosine similarity between the vector representation of the input and the vector representations of each sentence in the text
    similarity_scores = []
    for vector in vectors_list:
        score = query_vector.dot(vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
        similarity_scores.append(score)

    # Sort the sentences in descending order of their cosine similarity to the input and return the top-N most similar sentences
    n = 100
    most_similar_sentences = [[vectors_list[idx],idx] for idx in np.argsort(similarity_scores)[::-1][:n] if np.sum(vectors_list[idx]) != 0]

    return most_similar_sentences[:20]

## Prompt from user

In [17]:
userPrompt = "la mayor utilidad de la inteligencia artificial"
tokenized_prompt = preprocess_text_lemma(userPrompt)
print(tokenized_prompt)

promptVector_lemma = np.zeros((lemmaModel.vector_size,))
word_count = 0

for token in tokenized_prompt:
    if token in lemmaModel.wv.key_to_index:
        promptVector_lemma += lemmaModel.wv[token]
        word_count += 1
        print(token)

if word_count > 0:
    promptVector_lemma /= word_count
    
print(promptVector_lemma)

['utilidad', 'inteligencia', 'artificial']
utilidad
inteligencia
artificial
[-0.00952819  0.00466161 -0.02663288  0.01963261 -0.00161949  0.00766898
  0.00097638  0.01894364 -0.00940083  0.03277215  0.01118401 -0.00221441
  0.01903289  0.00620929  0.03384846  0.02461459 -0.00759683 -0.01435977
 -0.02247607 -0.00520005 -0.00277442 -0.01049296  0.01632351 -0.01474958
 -0.0177014  -0.01750472 -0.0079758   0.03869421 -0.01293144 -0.00072562
  0.02096445 -0.01934543 -0.00395717 -0.0252096   0.00152048  0.01453691
  0.0406304  -0.03561086 -0.02577366 -0.03884867]


In [18]:
var = cosine_similarity_list(df_esp['vector'],promptVector_lemma)

In [19]:
var=np.array(var)
var[:,1]
possible_solutions=df_esp.iloc[var[:,1]]

C:\Users\afrincon\AppData\Local\Temp\ipykernel_3584\2543073143.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  var=np.array(var)


In [20]:
for paragraph in possible_solutions["content"]:
    print(paragraph, "\n")

La mayor virtud y utilidad de la inteligencia artificial no es lo que nos venden las películas, robots andantes que aprenden de nuestros movimientos para hacer una revolución. La inteligencia artificial, es lo que nos permite ser más eficientes cada instante, nos permite ser más asertivos al tomar decisiones, incluso más importante nos permite tener una mejor . 

La mayor virtud y utilidad de la inteligencia artificial no es lo que nos venden las películas, robots andantes que aprenden de nuestros movimientos para hacer una revolución. La inteligencia artificial, es lo que nos permite ser más eficientes cada instante, nos permite ser más asertivos al tomar decisiones, incluso más importante nos permite tener una mejor experiencia como humanos. Cuando se desarrolla inteligencia artificial no solo se debe hacer en un esquema ético y legal, sino también se debe hacer con el propósito de complementar la inteligencia humana. Los humanos tenemos una capacidad mental inigualable, en últimas s